In [ ]:
!cea --help 

usage: cea SCRIPT [OPTIONS]
       to run a specific script
usage: cea --help SCRIPT
       to get additional help specific to a script

SCRIPT can be one of:
[Data management]:  archetypes-mapper, streets-helper,
    surroundings-helper, terrain-helper, trees-helper, weather-helper
[Demand forecasting]:  demand, radiation, radiation-simplified,
    schedule-maker
[Energy potentials]:  photovoltaic, photovoltaic-thermal,
    sewage-potential, shallow-geothermal-potential, solar-collector,
    water-body-potential
[Life cycle analysis]:  emissions, system-costs
[Networks]:  network-layout, thermal-network
[Optimization]:  decentralized, optimization-new,
    thermal-network-optimization
[Utilities]:  batch-process-workflow, create-mixed-use-type,
    data-initializer, data-migrator, dbf-to-csv-to-dbf,
    rename-building, run-all-plots, run-unit-tests,
    sensitivity-analysis-sampler, shp-to-csv-to-shp, test
[default]:  compile, create-polygon, dashboard, excel-to-shapefile,
    extrac

hey, here i will try to describe each step to get to a non-stop simulation of multiple scenarios.
for now there will be a lot of manual steps and procedures; they will be made obsolet step by step

Start by having a Base Scenario
for example: C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe

simple functions necessary to get to a demand simulation, each with the available inputs

--archetypes-mapper\
--scenario C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe\
--input-databases ['comfort', 'architecture', 'air-conditioning', 'internal-loads', 'supply', 'schedules']\
--buildings []


--radiation\
--scenario C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe\
--multiprocessing True\
--number-of-cpus-to-keep-free 1\
--debug False\
--buildings []\
--use-latest-daysim-binaries True\
--albedo 0.2\
--roof-grid: 10\
--walls-grid: 200\
--zone-geometry: 2.0\
--surrounding-geometry: 5.0\
--consider-floors: True\
--neglect-adjacent-buildings: False\
--rad-ab: 4\
--rad-ad: 512\
--rad-as: 32\
--rad-ar: 20\
--rad-aa: 0.15\
--rad-lr: 8\
--rad-st: 0.5\
--rad-sj: 0.7\
--rad-lw: 0.05\
--rad-dj: 0.7\
--rad-ds: 0.0\
--rad-dr: 0\
--rad-dp: 32\
--daysim-bin-directory:
--n-buildings-in-chunk: 100\
--write-sensor-data: True

--schedule-maker\
--scenario C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe\
--multiprocessing True\
--number-of-cpus-to-keep-free 1\
--debug False\
--buildings []\
--schedule-model deterministic

--demand\
--scenario: C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe
--multiprocessing: True
--number-of-cpus-to-keep-free: 1
--debug: False
--buildings: []
--loads-output: []
--massflows-output: []
--temperatures-output: []
--resolution-output: hourly
--use-dynamic-infiltration-calculation: False
--overheating-warning: True

hier Anmerkugn an mich selber!
TRY Auswirkungen durch dynamic infiltration und overheating warning

In [ ]:
#!cea --archetypes-mapper
#!cea --radiation
#!cea --schedule-maker
#!cea --demand

In [12]:
def run_cea_command(command):
    """Runs a given CEA command."""
    
    # Command to execute the CEA command (assuming the environment is activated)
    !cea {command}


In [ ]:
def run_cea_schedule_maker(scenario):
    !cea schedule-maker --scenario {scenario} --multiprocessing True --number-of-cpus-to-keep-free 1 --debug False --buildings [] --schedule-model deterministic

def run_cea_archetypes_mapper(scenario):
    !cea archetypes-mapper --scenario {scenario} --input-databases ['comfort', 'architecture', 'air-conditioning', 'internal-loads', 'supply', 'schedules'] --buildings []

def run_cea_solar_radiation(scenario):
    !cea radiation --scenario {scenario} --multiprocessing True --number-of-cpus-to-keep-free 1 --debug False --buildings [] --use-latest-daysim-binaries True --albedo 0.2\
    --roof-grid 10 --walls-grid 200 --zone-geometry 2.0 --surrounding-geometry 5.0 --consider-floors True --neglect-adjacent-buildings False --rad-ab 4 --rad-ad 512 --rad-as 32\
    --rad-ar 20 --rad-aa 0.15 --rad-lr 8 --rad-st 0.5 --rad-sj 0.7 --rad-lw 0.05 --rad-dj 0.7 --rad-ds 0.0 --rad-dr 0 --rad-dp 32 --daysim-bin-directory --n-buildings-in-chunk 100\
    --write-sensor-data True

def run_cea_demand(scenario):
    !cea demand --scenario {scenario} --multiprocessing True --number-of-cpus-to-keep-free 1 --debug False --buildings [] --loads-output [] --massflows-output [] --temperatures-output []\
    --resolution-output hourly --use-dynamic-infiltration-calculation False --overheating-warning True

In [ ]:
def run_simulation_workflow(scenario_path, weather_file):
    # Step 1: Set up and run archetypes-mapper
    print("Running 'archetypes-mapper'...")
    run_cea_command_in_env(f"archetypes-mapper --scenario={scenario_path}")
    
    # Step 2: Run radiation and schedule-maker sequentially
    print("Running 'radiation'...")
    run_cea_command_in_env(f"radiation --scenario={scenario_path}")
    
    print("Running 'schedule-maker'...")
    run_cea_command_in_env(f"schedule-maker --scenario={scenario_path}")
    
    # Step 3: Run the demand script after the others
    print("Running 'demand'...")
    run_cea_command_in_env(f"demand --scenario={scenario_path}")

# Example usage with a single scenario and weather file
run_simulation_workflow("C:/scenario/baseline", "Zurich")


so werden neue typologys zugewiesen bzw. in "neue" scenarien geladen

In [42]:
!cea dbf-to-csv-to-dbf --scenario "C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe_ph" --input-file "C:\Users\ma0026.NB-MA0026\Downloads\typologies.csv"  --output-file-name "typology.dbf" --output-path "C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe_ph\inputs\building-properties"

City Energy Analyst version 3.38.0
Running `cea dbf-to-csv-to-dbf` with the following parameters:
- general:scenario = C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe_ph
  (default: {general:project}\{general:scenario-name})
- dbf-tools:input-file = C:\Users\ma0026.NB-MA0026\Downloads\typologies.csv
  (default: )
- dbf-tools:output-file-name = typology.dbf
  (default: converted_file.dbf)
- dbf-tools:output-path = C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe_ph\inputs\building-properties
  (default: c:\Users\ma0026.NB-MA0026\Documents\Code\ceautil\ceautil)
Execution time: 1.07s


C:\Users\ma0026.NB-MA0026\Documents\Github\CityEnergyAnalyst\cea\utilities\dbf.py:80: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(input_file, sep=None)


In [45]:
import subprocess
import cea.config
import time

def run_cea_command(script_name, **kwargs):
    """Utility function to run a CEA command with optional parameters."""
    command = ["cea", script_name]
    for key, value in kwargs.items():
        command.append(f"--{key}={value}")
    
    try:
        subprocess.run(command, check=True)
        print(f"'{script_name}' script completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error running '{script_name}' script: {e}")

def run_simulation_workflow(scenario_path):
    # Step 1: Load and set up the configuration file for the scenario
    config = cea.config.Configuration()
    config.scenario = scenario_path
    #config.weather = weather_file
    config.save()
    
    # Step 2: Run the 'archetypes-mapper' script
    print("Starting 'archetypes-mapper'...")
    run_cea_command("archetypes-mapper", scenario=scenario_path)

    # Step 3: Run 'radiation' and 'schedule-maker' in sequence after 'archetypes-mapper'
    print("Starting 'radiation'...")
    run_cea_command("radiation", scenario=scenario_path)
    
    print("Starting 'schedule-maker'...")
    run_cea_command("schedule-maker", scenario=scenario_path)

    # Step 4: Run the 'demand' script after both previous scripts are completed
    print("Starting 'demand'...")
    run_cea_command("demand", scenario=scenario_path)

# Example usage
scenarios = [r"C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe", r"C:\Users\ma0026.NB-MA0026\Documents\CEA_Projekte\gebaeudemodelle\Seestadt_Reihe_ph"]
#weather_files = ["Zurich.epw", "Berlin.epw"]

for scenario in scenarios:
    run_simulation_workflow(scenario)


Starting 'archetypes-mapper'...
Error running 'archetypes-mapper' script: Command '['cea', 'archetypes-mapper', '--scenario=C:\\Users\\ma0026.NB-MA0026\\Documents\\CEA_Projekte\\gebaeudemodelle\\Seestadt_Reihe']' returned non-zero exit status 1.
Starting 'radiation'...
Error running 'radiation' script: Command '['cea', 'radiation', '--scenario=C:\\Users\\ma0026.NB-MA0026\\Documents\\CEA_Projekte\\gebaeudemodelle\\Seestadt_Reihe']' returned non-zero exit status 1.
Starting 'schedule-maker'...
Error running 'schedule-maker' script: Command '['cea', 'schedule-maker', '--scenario=C:\\Users\\ma0026.NB-MA0026\\Documents\\CEA_Projekte\\gebaeudemodelle\\Seestadt_Reihe']' returned non-zero exit status 1.
Starting 'demand'...
Error running 'demand' script: Command '['cea', 'demand', '--scenario=C:\\Users\\ma0026.NB-MA0026\\Documents\\CEA_Projekte\\gebaeudemodelle\\Seestadt_Reihe']' returned non-zero exit status 1.
Starting 'archetypes-mapper'...
Error running 'archetypes-mapper' script: Command 